In [ ]:
%load_ext autoreload
%autoreload 2

import torch.cuda

from ultralytics import YOLO
from omegaconf import OmegaConf

from drone_detection.config import (
    DEMO_DATA_DIR,
    MODELS_DIR,
    CONFIG_DIR,
)
from drone_detection.models_utils import evaluate_model_video, open_web_camera_with_model

In [ ]:
# Supported video extensions (That's not all!)
video_file_extensions = (".mp4", ".avi",)

In [ ]:
config_path = CONFIG_DIR / "inference.yaml"
cfg = OmegaConf.load(config_path)

In [ ]:
# Data should be in the path ./drone-detection/data/demo_data/
data_path = DEMO_DATA_DIR / cfg.name_data
model_path = MODELS_DIR / cfg.name_model

device = "cuda" if torch.cuda.is_available() else "cpu"
OmegaConf.update(cfg, "params_eval.device", device)
OmegaConf.update(cfg, "params_tracking.device", device)

In [ ]:
model = YOLO(model_path)

In [ ]:
if cfg.camera:
    for bbox in open_web_camera_with_model(model, cfg.resize_frame, cfg.params_tracking):
        pass
elif data_path.suffix.endswith(video_file_extensions):
    for bbox in evaluate_model_video(model, data_path, cfg.resize_frame, cfg.params_tracking):
        pass
else:
    results = model.predict(data_path, **cfg.params_eval)
    results[0].show()